In [2]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('raw_sales_data.csv')
df.head()

,sale_id,product_id,product_name,category,quantity_sold,price_per_unit,sale_date,region
0,1,101,Laptop Pro,Electronics,5,1200.0,2023-01-15,North
1,2,102,Desktop Max,Electronics,2,1500.0,2023-01-16,North
2,3,201,Office Chair,Furniture,10,150.0,2023-01-17,South
3,4,103,Wireless Mouse,Electronics,25,25.0,2023-01-18,West
4,5,301,Python Programming Book,Books,50,45.0,2023-01-18,East


In [ ]:
# Add total_revenue column
df['total_revenue'] = df['quantity_sold'] * df['price_per_unit']

# Filter only Electronics category
df = df[df['category'] == 'Electronics']

# Format sale_date
df['sale_date'] = pd.to_datetime(df['sale_date']).dt.strftime('%Y-%m-%d')

# Reorder columns
df = df[['sale_id', 'product_name', 'category', 'total_revenue', 'sale_date', 'region']]
df.head()

,sale_id,product_name,category,total_revenue,sale_date,region
0,1,Laptop Pro,Electronics,6000.0,2023-01-15,North
1,2,Desktop Max,Electronics,3000.0,2023-01-16,North
3,4,Wireless Mouse,Electronics,625.0,2023-01-18,West
5,6,Laptop Pro,Electronics,3600.0,2023-01-19,South
7,8,Keyboard Pro,Electronics,1125.0,2023-01-20,East


In [ ]:
# Save the transformed data to a new CSV file
df.to_csv('processed_electronics_sales.csv', index=False)
